## simulation user lifetimes 

In [1]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_username__c'
object_id = 'Contact'

# login parameters to be handled by Papermill
sql_login_params = {"username": "wang2506_ro", "password": "fnVnwcCS7iT45EsA"}
sf_login_params = {
    "grant_type": "password",
    "client_id": "3MVG95jctIhbyCppj0SNJ75IsZ1y8UPGZtSNF4j8FNVXz.De8Lu4jHm3rjRosAtsHy6qjHx3i4S_QbQzvBePG",
    "client_secret": "D1623C6D3607D4FC8004B92C761DFB6C1F70CCD129C5501E357028DFA00F5764",
    "username":"wang2506@purdue.edu",
    "password":"sf2021shitOPmlIiFMLnrudgC6oSX0WV1T",   
} 

In [2]:
import sys
sys.path.append('/home/users/wang2506/nanohub_salesforce_integ/salesforce')
    
import pandas as pd
import time
import datetime
import sqlalchemy as sql

from DB2SalesforceAPI import DB2SalesforceAPI

In [3]:
## read in data
import os
cwd = os.getcwd()
clusters_df = pd.read_csv(cwd+'/manual_data_filters/clustered_users.csv')
selfs_df = pd.read_csv(cwd+'/manual_data_filters/self_study_users.csv')
researchers_df = pd.read_csv(cwd+'/manual_data_filters/researchers.csv')

In [4]:
clusters_df.head(5)

,Unnamed: 0,username
0,10813,0.yao.yuan
1,10817,008dilip
2,10821,01024461660a
3,10826,01luiyi
4,10828,0214moon


In [5]:
selfs_df.head(5)

,Unnamed: 0,username
0,0,-100005
1,1,-100024
2,2,-100026
3,3,-100032
4,4,-10006


In [6]:
researchers_df.head(5)

,Unnamed: 0,usernames
0,0,hspwong
1,1,onayfeh
2,2,accelrys
3,3,himadri
4,4,fossum


In [35]:
sql_query = "select min(datetime), max(datetime), user from toolstart group by user"
engine = sql.create_engine('mysql+pymysql://%s:%s@127.0.0.1/nanohub_metrics' \
                                               %('wang2506_ro', 'fnVnwcCS7iT45EsA'))

In [36]:
lifetimes_df = pd.read_sql_query(sql_query,engine)

In [96]:
lifetimes_df.head(5)

,min(datetime),max(datetime),user
0,0000-00-00 00:00:00,2013-05-28 15:46:14,
1,2017-03-08 10:15:23,2017-03-09 05:09:22,0.yao.yuan
2,2016-09-28 20:27:13,2016-10-12 23:50:56,008dilip
3,2013-11-19 00:53:23,2013-11-19 00:58:56,00ff
4,2017-03-20 09:46:36,2017-03-20 09:46:36,00thamizharasi00


In [97]:
lifetimes_df.shape

(146785, 3)

In [98]:
# remove garbage usernames
temp = lifetimes_df[lifetimes_df['user'] != '']

In [99]:
temp

,min(datetime),max(datetime),user
1,2017-03-08 10:15:23,2017-03-09 05:09:22,0.yao.yuan
2,2016-09-28 20:27:13,2016-10-12 23:50:56,008dilip
3,2013-11-19 00:53:23,2013-11-19 00:58:56,00ff
4,2017-03-20 09:46:36,2017-03-20 09:46:36,00thamizharasi00
5,2020-09-23 18:14:20,2020-12-11 02:58:30,01024461660a
...,...,...,...
146780,2016-07-13 00:40:03,2016-07-13 06:33:44,zzzstas
146781,2002-07-23 02:52:49,2002-07-23 03:04:24,z_depth
146782,2006-10-16 20:25:49,2006-10-16 20:27:53,Z_E_U_S
146783,2013-02-11 14:42:41,2013-02-19 01:53:39,z_henry35


In [100]:
lifetimes_df['min(datetime)'].to_numpy()[0]

'0000-00-00 00:00:00'

In [101]:
# replace 0000-00-00 with 2000-01-01
temp2 = lifetimes_df['min(datetime)'].to_list()
for i,j in enumerate(temp2):
    if type(j) == str:
        if j == '0000-00-00 00:00:00':
            temp2[i] = '2000-01-01 00:10:10'


In [102]:
len(temp2)

146785

In [103]:
temp2[0] == '0000-00-00 00:00:00'

False

In [104]:
temp['min(datetime)'] = temp2[1:]
temp['min(datetime)'] = pd.to_datetime(temp['min(datetime)'])
temp['max(datetime)'] = pd.to_datetime(temp['max(datetime)'])

C:\Users\ch5b2\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\ch5b2\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\ch5b2\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [129]:
## calculate lifetimes
lifetime_num = []
min_dates = temp['min(datetime)'].to_list()
for i,j in enumerate(temp['max(datetime)'].to_list()):
    temp_time = j-min_dates[i]
    lifetime_num.append(temp_time.days)


In [130]:
lifetime_num[:5]

[0, 14, 0, 0, 78]

In [131]:
temp['lifetimes'] = lifetime_num

C:\Users\ch5b2\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [132]:
display(temp.head(5))
display(temp.tail(5))

,min(datetime),max(datetime),user,lifetimes
1,2017-03-08 10:15:23,2017-03-09 05:09:22,0.yao.yuan,0
2,2016-09-28 20:27:13,2016-10-12 23:50:56,008dilip,14
3,2013-11-19 00:53:23,2013-11-19 00:58:56,00ff,0
4,2017-03-20 09:46:36,2017-03-20 09:46:36,00thamizharasi00,0
5,2020-09-23 18:14:20,2020-12-11 02:58:30,01024461660a,78


,min(datetime),max(datetime),user,lifetimes
146780,2016-07-13 00:40:03,2016-07-13 06:33:44,zzzstas,0
146781,2002-07-23 02:52:49,2002-07-23 03:04:24,z_depth,0
146782,2006-10-16 20:25:49,2006-10-16 20:27:53,Z_E_U_S,0
146783,2013-02-11 14:42:41,2013-02-19 01:53:39,z_henry35,7
146784,2008-07-09 22:51:51,2008-07-23 09:39:34,z_sath,13


In [133]:
from copy import deepcopy

In [134]:
send_df = deepcopy(temp)

In [135]:
send_df = send_df.rename(columns={'user':'nanoHUB_username__c','lifetimes':'sim_lifetimes__c',\
                    'min(datetime)':'earliest_sim__c','max(datetime)':'latest_sim__c'})

In [136]:
send_df

,earliest_sim__c,latest_sim__c,nanoHUB_username__c,sim_lifetimes__c
1,2017-03-08 10:15:23,2017-03-09 05:09:22,0.yao.yuan,0
2,2016-09-28 20:27:13,2016-10-12 23:50:56,008dilip,14
3,2013-11-19 00:53:23,2013-11-19 00:58:56,00ff,0
4,2017-03-20 09:46:36,2017-03-20 09:46:36,00thamizharasi00,0
5,2020-09-23 18:14:20,2020-12-11 02:58:30,01024461660a,78
...,...,...,...,...
146780,2016-07-13 00:40:03,2016-07-13 06:33:44,zzzstas,0
146781,2002-07-23 02:52:49,2002-07-23 03:04:24,z_depth,0
146782,2006-10-16 20:25:49,2006-10-16 20:27:53,Z_E_U_S,0
146783,2013-02-11 14:42:41,2013-02-19 01:53:39,z_henry35,7


In [138]:
db_s = DB2SalesforceAPI(sf_login_params)

Obtained Salesforce access token ...... True


In [139]:
db_s.external_id = external_id
db_s.object_id = object_id

In [140]:
db_s.send_data(send_df)

[Success] Bulk job creation successful. Job ID = 7505w00000UBoazAAD
hello
[Success] CSV upload successful. Job ID = 7505w00000UBoazAAD
[Success] Closing job successful. Job ID = 7505w00000UBoazAAD


In [159]:
db_s.check_bulk_status()

{'id': '7505w00000UBoazAAD',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-02-13T22:16:44.000+0000',
 'systemModstamp': '2021-02-13T22:22:23.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_username__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 146784,
 'numberRecordsFailed': 784,
 'retries': 0,
 'totalProcessingTime': 1423096,
 'apiActiveProcessingTime': 1277568,
 'apexProcessingTime': 10870041}

In [147]:
import pprint
pprint.pprint(db_s.check_bulk_failed_results())

''
